## Facial Expressions Recognition using Deep Convolutional Neural Networks
#### Seminar<br/>Computer Science and Engineering<br/>SJCE<br/>Mysuru


### Pre-processing and Training

In [2]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [3]:
num_classes = 7
batch_size = 256
epochs = 16

In [4]:
with open("/home/pradeep/Downloads/all/fer2013/fer2013.csv") as f:
    content = f.readlines()

In [5]:
lines = np.array(content)
num_of_instances = lines.size
print("Number of instances : ", num_of_instances)
print("Instance Length : ",len(lines[1].split(",")[1].split(" ")))

Number of instances :  35888
Instance Length :  2304


#### Preprocessing the dataset

In [7]:
x_train,y_train,x_test,y_test = [],[],[],[]
for i in range(1,num_of_instances):
    try:
        emotion,img,usage = lines[i].split(",")
        val = img.split(" ")
        pixels = np.array(val,'float32')
        emotion = keras.utils.to_categorical(emotion,num_classes)
        
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("E",end="")

In [8]:
x_train = np.array(x_train,'float32')
y_train = np.array(y_train,'float32')
x_test = np.array(x_test,'float32')
y_test = np.array(y_test,'float32')

In [9]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(28709, 2304)
(28709, 7)
(3589, 2304)
(3589, 7)


In [10]:
x_train /= 255
x_test /= 255

In [11]:
x_train = x_train.reshape(x_train.shape[0],48,48,1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0],48,48,1)
x_test = x_test.astype('float32')

### Construction of CNN Architecture

In [12]:
model = Sequential()

#### 1st convolutional layer

In [13]:
model.add(Conv2D(32,(5,5),activation = 'relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

#### 2nd convolutional layer

In [14]:
model.add(Conv2D(32,(4,4),activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3),strides=(2,2)))

#### 3rd convolutional layer

In [15]:
model.add(Conv2D(64,(5,5),activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3),strides=(2,2)))

#### Fully connected layer

In [16]:
model.add(Flatten())

In [17]:
# fully connected NN
model.add(Dense(3072,activation='relu'))
model.add(Dropout(0.2))

In [18]:
model.add(Dense(num_classes,activation='softmax'))

In [19]:
gen = ImageDataGenerator()
train_generator = gen.flow(x_train,y_train,batch_size=batch_size)

In [22]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 44, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 32)        16416     
_________________________________________________________________
average_pooling2d_1 (Average (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          51264     
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
__________

In [23]:
model.fit_generator(train_generator,steps_per_epoch=batch_size,epochs=epochs)

Epoch 1/16
256/256 [==============================] - 218s 850ms/step - loss: 1.7929 - acc: 0.2582
Epoch 2/16
256/256 [==============================] - 192s 749ms/step - loss: 1.6477 - acc: 0.3399
Epoch 3/16
256/256 [==============================] - 191s 746ms/step - loss: 1.5038 - acc: 0.4193
Epoch 4/16
256/256 [==============================] - 191s 746ms/step - loss: 1.4125 - acc: 0.4575
Epoch 5/16
256/256 [==============================] - 192s 750ms/step - loss: 1.3412 - acc: 0.4869
Epoch 6/16
256/256 [==============================] - 191s 746ms/step - loss: 1.2916 - acc: 0.5078
Epoch 7/16
256/256 [==============================] - 192s 750ms/step - loss: 1.2431 - acc: 0.5288
Epoch 8/16
256/256 [==============================] - 191s 746ms/step - loss: 1.2134 - acc: 0.5389
Epoch 9/16
256/256 [==============================] - 191s 746ms/step - loss: 1.1775 - acc: 0.5569
Epoch 10/16
256/256 [==============================] - 191s 748ms/step - loss: 1.1499 - acc: 0.5666
Epoch 11/

### Saving the model

In [24]:
model.save('FERModel2.h5')